In [3]:
!pip install pandas matplotlib seaborn

  Using cached matplotlib-3.9.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.0-cp312-cp312-win_amd64.whl.metadata (167 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.8 MB 524.3 kB/s eta 0:00:14
   -- ------------------------------------- 0.5/7.8 MB 524.3 kB/s eta 0:00:14
   ---- ----------------------------------- 0.8/7.8 MB 621.2 kB/s eta 0:00:12
   ----- ---------------------------------- 1.0/7.8 MB 

In [14]:
!pip install pyspark

  Using cached pyspark-3.5.3.tar.gz (317.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840672 sha256=fe1746eb982c2e9398ca7e432c60e8f3e24cbfb2159e50e575aa2c00a03a46df
  Stored in directory: c:\users\acer\appdata\local\pip\cache\wheels\07\a0\a3\d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark


In [5]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, lit
spark = SparkSession.builder.appName("Covid-19 Data Preprocessing").getOrCreate()
df = spark.read.csv('D:/Pyspark_hackathon/dataset.csv',header = True, inferSchema=True)
print("Initial Schema")
# df.printSchema()
# df.show(5)
# Iterate over schema and print column names and types
for field in df.schema.fields:
    print(f"Column Name: {field.name}, Data Type: {field.dataType}")


Initial Schema
Column Name: country, Data Type: StringType()
Column Name: date, Data Type: DateType()
Column Name: total_cases, Data Type: IntegerType()
Column Name: new_cases, Data Type: IntegerType()
Column Name: new_cases_smoothed, Data Type: DoubleType()
Column Name: total_cases_per_million, Data Type: DoubleType()
Column Name: new_cases_per_million, Data Type: DoubleType()
Column Name: new_cases_smoothed_per_million, Data Type: DoubleType()
Column Name: total_deaths, Data Type: IntegerType()
Column Name: new_deaths, Data Type: IntegerType()
Column Name: new_deaths_smoothed, Data Type: DoubleType()
Column Name: total_deaths_per_million, Data Type: DoubleType()
Column Name: new_deaths_per_million, Data Type: DoubleType()
Column Name: new_deaths_smoothed_per_million, Data Type: DoubleType()
Column Name: excess_mortality, Data Type: DoubleType()
Column Name: excess_mortality_cumulative, Data Type: DoubleType()
Column Name: excess_mortality_cumulative_absolute, Data Type: DoubleType()


In [13]:
from pyspark.sql import functions as F
null_counts  = df.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df.columns])
null_counts.show()

+-------+----+-----------+---------+------------------+-----------------------+---------------------+------------------------------+------------+----------+-------------------+------------------------+----------------------+-------------------------------+----------------+---------------------------+------------------------------------+---------------------------------------+-------------+-------------------------+----------------------+----------------------------------+------------+------------------------+---------------------+---------------------------------+----------------+-----------------+-----------+---------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+------------------+-----------------+-----------------------+--------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+--

In [ ]:
data_without_null= null_counts.dropna()

In [12]:
data_without_null

DataFrame[country: string, date: date, total_cases: int, new_cases: int, new_cases_smoothed: double, total_cases_per_million: double, new_cases_per_million: double, new_cases_smoothed_per_million: double, total_deaths: int, new_deaths: int, new_deaths_smoothed: double, total_deaths_per_million: double, new_deaths_per_million: double, new_deaths_smoothed_per_million: double, excess_mortality: double, excess_mortality_cumulative: double, excess_mortality_cumulative_absolute: double, excess_mortality_cumulative_per_million: double, hosp_patients: int, hosp_patients_per_million: double, weekly_hosp_admissions: int, weekly_hosp_admissions_per_million: double, icu_patients: int, icu_patients_per_million: double, weekly_icu_admissions: int, weekly_icu_admissions_per_million: double, stringency_index: double, reproduction_rate: double, total_tests: bigint, new_tests: int, total_tests_per_thousand: double, new_tests_per_thousand: double, new_tests_smoothed: int, new_tests_smoothed_per_thousand: